In [4]:
%reload_ext autoreload
%autoreload 2
import ast
import pickle
import itertools
from collections import Counter
from tqdm import tqdm
import pandas as pd
import os

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import cv2
import swifter

from analysis.generate_cluster_information_file import load_scored_info, extract_all_information_query, to_df_query
from common.image_processing import pixel_intensity_histogram


from sklearn.model_selection import cross_val_score
from sklearn.ensemble import IsolationForest, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
import xgboost
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [5]:
train_X = pd.read_csv('train_X.csv')
train_y = pd.read_csv('train_y.csv')

In [6]:
train_X.head()

,cluster_num,background_threshold,cluster_num_intensities_avg,cluster_num_intensities_max,cluster_num_intensities_min,cluster_num_intensities_std,cluster_peak_intensities_avg,cluster_peak_intensities_max,cluster_peak_intensities_min,cluster_peak_intensities_std,...,largest_clusters_distance,brightest_clusters_distance,num_large_clusters,num_small_clusters,num_bright_clusters,num_dark_clusters,num_large_bright_clusters,num_small_bright_clusters,num_large_dark_clusters,num_small_dark_clusters
0,50.0,0.0,6.680000,71.0,1.0,13.844046,14.920000,255.0,1.0,49.370777,...,418.150691,418.150691,42.0,8.0,17.0,33.0,17.0,0.0,25.0,8.0
1,104.0,0.0,5.692308,61.0,1.0,11.164915,11.375000,255.0,1.0,36.751030,...,537.729486,605.561723,84.0,20.0,27.0,77.0,27.0,0.0,57.0,20.0
2,71.0,1.0,5.605634,84.0,1.0,12.496173,11.563380,255.0,2.0,34.161890,...,758.641549,841.503417,59.0,12.0,24.0,47.0,24.0,0.0,35.0,12.0
3,49.0,2.0,8.428571,69.0,1.0,13.074356,22.612245,255.0,3.0,51.612297,...,798.068919,257.670332,43.0,6.0,33.0,16.0,33.0,0.0,10.0,6.0
4,169.0,0.0,3.284024,47.0,1.0,6.639654,6.621302,255.0,1.0,26.394791,...,1064.421439,137.116739,115.0,54.0,28.0,141.0,28.0,0.0,87.0,54.0


In [7]:
scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_X)
train_y_scaled = scaler.fit_transform(train_y)
train_X_na = train_X.fillna(0)
train_X_na_scaled = scaler.fit_transform(train_X_na)

### Tuning XGBoost

In [8]:
"""
    Tunes hyperparameters for xgbosot
"""
hyperparameters = { 
    'n_estimators': [100, 500, 1000, 5000],
    'learning_rate': [0.01, 0.05, 0.1, 0.5, 0.7],
    'max_depth': [3, 6, 8],
    'booster': ['gbtree', 'gblinear'],
    'subsample': [0.5, 0.7, 1],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'gamma': [0, 0.1, 0.5]
}

clf = RandomizedSearchCV(xgboost.XGBRegressor(objective='reg:squarederror'), 
                         hyperparameters, scoring=make_scorer(mean_absolute_error),
                         random_state=1, n_iter=100, cv=10, verbose=1, n_jobs=-1)
clf.fit(train_X_scaled, train_y)
clf.__dict__

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


KeyboardInterrupt: 

In [18]:
print(clf.best_estimator_)
print('best score: ', clf.best_score_)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1.0, gamma=0.5, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.01, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.7, tree_method=None,
             validate_parameters=False, verbosity=None)
best score:  0.4477113293611311


In [ ]:
"""
    Tunes hyperparameters for RandomForest
"""
hyperparameters = { 
    'n_estimators': [100, 500, 1000, 5000],
    'criterion' : ['mse', 'mae'],
    'max_depth': [3, 6, 8, 20, None],
    'min_samples_split' : [2, 4, 8, 16],
    'min_samples_leaf' : [1, 2, 4],
    'max_features' : ['auto', 'sqrt', 'log2']
}

randomforest = RandomForestRegressor()

clfRF = RandomizedSearchCV(randomforest, 
                         hyperparameters, 
                         random_state=1, n_iter=100, cv=10, verbose=2, n_jobs=-1)
clfRF.fit(train_X_na_scaled, np.ravel(train_y_scaled))
clfRF.__dict__

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  6.4min


In [6]:
randomforest = RandomForestRegressor()
randomforest.fit(train_X_na_scaled, np.ravel(train_y_scaled))

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)